### Imports

In [2]:
# Install proper version of torch, as according to: https://pytorch.org/
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
!pip install -q -e ../.

import pandas as pd
import torch
from looptune import prep_config_combinations, single_run, clean_memory

### Prepare dataset with two columns: 'text' and 'label'

Examplary data source: https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news


In [2]:
df = pd.read_csv('example_data/SentimentAnalysisforFinancialNews.csv', encoding="ISO-8859-1", header=None)
df.columns = ['label', 'text']
df.head(5)

,label,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


### Prepare run configurations

In [17]:
run_config = {   # -----------------------
                 'model_name': 'meta-llama/Meta-Llama-3-8B', # Pre-trained model names from the Hugging Face hub used for fine-tuning
                 # --------------------------
                 'split': (0.7, 0.3), # Divides the dataset into training, testing, (and optionally) validation sets. Examples: (0.7,0.3) -> split into train and test proportionally; (70, 30) splits into train,test proportionally.
                 'binary': False, # Indicates whether the task is binary (two classes) or multi-class classification.,
                 'balanced': (('train',), ('test',)),
                 # --------------------------
                 'training_arguments': {
                     'num_train_epochs': 1, # Number of times the model sees the entire training dataset.
                     'per_device_train_batch_size': 4, # Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                     'per_device_eval_batch_size': 4, # Number of samples processed in each evaluation step.
                     # 'gradient_accumulation_steps': 4,
                     'gradient_checkpointing': True,
                     #-----------------------------
                     'save_total_limit': 2,
                     'load_best_model_at_end': True,
                     'save_strategy': 'steps', # Controls when to save model checkpoints ('steps', 'epoch' or 'no').
                     'metric_for_best_model': 'f1-score',
                     #-----------------------------
                     'evaluation_strategy': "steps",
                     'logging_steps': 20,
                     'max_steps': 20,
                     'fp16': False,
                     # 'use_cpu': False,
                     #-----------------------------
                     'learning_rate': 5e-5,
                     'lr_scheduler_type': "linear",
                     'warmup_ratio': 0.1,
                     'max_grad_norm': 0.3,
                     'weight_decay': 0.001,
                 },
                 #-----------------------------
                     'bnb_config': [
                                # False,
                                {'bnb_4bit_compute_dtype': torch.bfloat16, 'load_in_4bit': True, 'bnb_4bit_quant_type': "nf4", 'bnb_4bit_use_double_quant': True, 'load_in_8bit': False}
                                 ],
                 'peft_config': [
                                # False,
                                {'r': 2, 'lora_alpha': 8, 'lora_dropout': 0.05, 'bias': "none",
                                'task_type': "SEQ_CLS", 
                                # 'target_modules': ("v_proj",),
                                'target_modules': "all-linear"
                                }
                                ],
                    }

run_params_serie = prep_config_combinations(run_config)
run_params_serie

[{'model_name': 'meta-llama/Meta-Llama-3-8B',
  'split': (0.7, 0.3),
  'binary': False,
  'balanced': (('train',), ('test',)),
  'training_arguments': {'num_train_epochs': 1,
   'per_device_train_batch_size': 4,
   'per_device_eval_batch_size': 4,
   'gradient_checkpointing': True,
   'save_total_limit': 2,
   'load_best_model_at_end': True,
   'save_strategy': 'steps',
   'metric_for_best_model': 'f1-score',
   'evaluation_strategy': 'steps',
   'logging_steps': 20,
   'max_steps': 20,
   'fp16': False,
   'learning_rate': 5e-05,
   'lr_scheduler_type': 'linear',
   'warmup_ratio': 0.1,
   'max_grad_norm': 0.3,
   'weight_decay': 0.001},
  'bnb_config': {'bnb_4bit_compute_dtype': torch.bfloat16,
   'load_in_4bit': True,
   'bnb_4bit_quant_type': 'nf4',
   'bnb_4bit_use_double_quant': True,
   'load_in_8bit': False},
  'peft_config': {'r': 2,
   'lora_alpha': 8,
   'lora_dropout': 0.05,
   'bias': 'none',
   'task_type': 'SEQ_CLS',
   'target_modules': 'all-linear'}}]

### Run in loop

In [22]:
for run_params in run_params_serie:
    single_run(run_params, df)

          text
label         
negative   604
neutral   2879
positive  1363


Casting to class labels:   0%|          | 0/4846 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1269 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/543 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
max_steps is given, it will override any value given in num_train_epochs


trainable params: 5,263,366 || all params: 7,510,208,524 || trainable%: 0.0701
GPU = NVIDIA GeForce RTX 3080. Max memory = 10.0 GB.
6.402 GB of memory reserved.


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
20,0.780900,nan,0.111111,0.333333,0.166667,0.333333


70.6655 seconds used for training.
1.18 minutes used for training.
Peak reserved memory = 6.402 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 64.02 %.
Peak reserved memory for training % of max memory = 0.0 %.
